## ✅ Roadmap del Proyecto de Web Scraping y Carga a Base de Datos

Este proyecto tiene como objetivo scrapear libros desde una web pública, obtener información adicional mediante una API externa y almacenar los datos en una base de datos relacional para su posterior análisis.

### 🛠️ Etapas del desarrollo

- [ ] **1. Web Scraping**
  - [x] Scrapear todos los géneros desde la página principal.
  - [x] Iterar sobre cada género para obtener todos los libros listados.
  - [ ] Extraer título, precio, stock, rating y link a la página del libro.
  - [ ] Extraer información detallada desde la página de cada libro (si es necesario).
  - [ ] Consultar la API de Google Books para obtener el autor y más detalles (usando el título).
  - [ ] Guardar los datos obtenidos en archivos `.csv` separados:
    - [ ] Libros (`libros.csv`)
    - [ ] Autores (`autores.csv`)
    - [ ] Géneros (`generos.csv`)

- [ ] **2. Validación**
  - [ ] Verificar la integridad y limpieza de los datos descargados.
  - [ ] Eliminar duplicados y manejar valores nulos si los hay.

- [ ] **3. Diseño de Base de Datos Relacional**
  - [ ] Crear script DDL:
    - [ ] Crear la base de datos (si no existe).
    - [ ] Crear las tablas `libros`, `autores`, `generos`, y tablas intermedias para relaciones N:N si aplica.
  - [ ] Crear diagrama UML/ER para visualizar relaciones.

- [ ] **4. Inserción de Datos**
  - [ ] Crear script DML para insertar los datos desde los CSV a la base de datos (usando Python + `psycopg2` o `SQLAlchemy` para PostgreSQL).
  - [ ] Comprobar inserciones correctas mediante queries de prueba.

- [ ] **5. Consultas y Análisis**
  - [ ] Escribir consultas SQL para:
    - [ ] Obtener todos los libros por género.
    - [ ] Buscar libros por autor.
    - [ ] Calcular estadísticas como precio promedio por género o autor.
  - [ ] (Opcional) Crear una vista para simplificar reportes.

---

> 📌 **Nota:** La base de datos será inicialmente verificada con SQLite por simplicidad, luego se adaptará a PostgreSQL para una implementación más robusta con PgAdmin.


In [ ]:
!pip install BeautifulSoup4
!pip install sqlalchemy psycopg2-binary pandas
!pip install requests


In [ ]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests #instalar
import time

base_url = "http://books.toscrape.com/"
url = base_url + "index.html"

#funcion get genero
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Lista de géneros
generos = soup.find("ul", class_="nav nav-list").find("ul").find_all("a")

for genero in generos:
    nombre = genero.get_text(strip=True)
    href = genero["href"]#?
    genero_url = base_url + "catalogue" + href

def obtener_urls_libros_por_genero(genero_url):
    urls_libros = []
    while genero_url:
        res = requests.get(genero_url)
        soup = BeautifulSoup(res.content, "html.parser")

        # Obtener links a los libros
        for li in soup.find_all("li", {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"}):
            a = li.find("a")
            href = a["href"]
            full_url = urljoin(genero_url, href)
            print(full_url)
            urls_libros.append(full_url)
            print(len(urls_libros))
            next_button = soup.find("li", class_="next")
        if next_button:
            next_href = next_button.find("a")["href"]
            genero_url = urljoin(genero_url, next_href)
        else:
            genero_url = None
    return urls_libros

#guardar en un diccionario los campos de los libros
def obtener_datos_libro(urls_libros):
    response = requests.get(urls_libros)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Título
    titulo = soup.h1.text.strip()

    # Precio
    precio_tag = soup.find('p', class_='price_color')
    precio = precio_tag.text.strip() if precio_tag else None

    # Rating
    rating_tag = soup.find('p', class_='star-rating')
    rating = rating_tag['class'][1] if rating_tag else 'No rating'

    # Descripción
    descripcion_tag = soup.select_one('#product_description ~ p')
    descripcion = descripcion_tag.text.strip() if descripcion_tag else 'No disponible'# me parece que no es necesario el if

    # Tabla de datos adicionales
    tabla = soup.find('table', class_='table table-striped')
    filas = tabla.find_all('tr')
    datos = {fila.th.text.strip(): fila.td.text.strip() for fila in filas}

    # Extracción específica
    upc = datos.get('UPC', '')#como funciona esto?
    tipo_producto = datos.get('Product Type', '')
    precio_excl_tax = datos.get('Price (excl. tax)', '')
    precio_incl_tax = datos.get('Price (incl. tax)', '')
    tax = datos.get('Tax', '')
    stock_raw = datos.get('Availability', '')
    numero_stock = ''.join(filter(str.isdigit, stock_raw))
    numero_reviews = datos.get('Number of reviews', '')

    return {
        'titulo': titulo,
        'precio': precio,
        'rating': rating,
        'descripcion': descripcion,
        'upc': upc,
        'tipo_producto': tipo_producto,
        'precio_excl_tax': precio_excl_tax,
        'precio_incl_tax': precio_incl_tax,
        'tax': tax,
        'stock': numero_stock,
        'numero_reviews': numero_reviews
    }

for genero in generos:
    nombre_genero = genero.get_text(strip=True)
    href = genero["href"]
    genero_url = urljoin(base_url, href)
    print(f"\nGénero: {genero_url}")
    urls_libros = obtener_urls_libros_por_genero(genero_url)
    for url_libro in urls_libros:
        print(f"\nLibro: {url_libro}")
        datos = obtener_datos_libro(url_libro)
        for k, v in datos.items():
            print(f"{k}: {v}")
        print("-" * 40)
        time.sleep(1)
     